In [1]:
reset()
%display latex
Partitions.options.latex="list"
load("setup.sage")

KT_vars = ['x','y','z1','z2','z3','z4','z5','v1','v2','v3','v4','v5']
KT_ring = LaurentPolynomialRing(QQ, KT_vars)

coho_vars = ['t','m','l1','l2','l3','l4','l5','w1','w2','w3','w4','w5']
coho_ring = LaurentPolynomialRing(QQ, coho_vars)


x,y,z1,z2,z3,z4,z5,v1,v2,v3,v4,v5=KT_ring.gens()
t,m,l1,l2,l3,l4,l5,w1,w2,w3,w4,w5= coho_ring.gens()
b=var('b')
'''   w=SR.var('w',5)   '''
v=[v1,v2,v3,v4,v5]
w=[w1,w2,w3,w4,w5]
L=var('l')
z=[z1,z2,z3,z4,z5]


def character(p,x=x,y=y):
    return sum(y^(-r) * x^(-c) for r, c in p.cells())

def character_vb(p, w,x=x,y=y):
    Q = character(p,x,y)
    return sum(Q * x^(w[2*i]) * y^(w[2*i+1]) for i in range(len(w)/2))

def dual(f):
    R = f.parent()
    g= R.zero()
    for exp, coeff in f.dict().items():
        g+= R({exp.emul(-1): coeff})
    return g

def tangent(p,x=x,y=y):
    total=0
    for r,c in p.cells():
        arm=p[r]-c-1
        leg=p.conjugate()[c]-r-1
        total+=y^(-leg)*x^(arm+1)+y^(leg+1)*x^(-arm)
    return total

def canonical_bundle(p,x=x,y=y):
    return character_vb(p, (-1,-1),x,y)


def structure_sheaf(p,x=x,y=y):
    return character_vb(p, (0,0),x,y)

def top_chern(f):
    numer, denom = [], []
    for coeff, mon in list(f):
        wt = from_monomial(mon)
        #if wt==0:
            #continue
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    
    return coho_ring.fraction_field()(prod(numer) / prod(denom))

def determinant(f):
    if f in ZZ:
        return 1
    else:
        return prod((mon)^coeff for coeff, mon in f)

def from_monomial(m):
    return sum(m.degree(KT_ring.gen(i)) * coho_ring.gen(i) for i in range(len(coho_ring.gens())))

def from_character(f):
    numer, denom = [], []

    for coeff, mon in list(f):
        wt = from_monomial(mon)
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    return numer, denom

def measure_unsymmetrized(f, inv=True):
    """
    computes 1/(1-x^-1) for K-thy class x. If inv=false, returns 1/(1-x)
    """
    R = f.parent()
    
    if f.parent() is ZZ:
        L = [(ZZ(f), 1)]
    else:
        L = list(f)

    numer, denom = R.one(), R.one()
    for coeff, monomial in L:
        if inv:
            term = 1 - monomial^-1
        else:
            term = 1 - monomial
        if monomial==1:
            term = 0
        if coeff < 0:
            numer *= term ** (-coeff)
        elif coeff > 0:
            denom *= term ** coeff
    return numer / denom



def degn(f,n,b=b):
    g=f.taylor(b,0,n+2)
    L=g.coefficients(b)
    for i in range(len(L)):
        if L[i][1]==n:
            return L[i][0]
    return 0

    

In [2]:
def partitions(Sum, K):
    '''computes fixed components of Quot by CC^* action.'''
    total=[]
    def ptn(Sum, K, lst, total):
        if K == 0:
            if Sum == 0:
                total+=[lst]
            return
        for i in range(0, min(Sum + 1, Sum + 1)):
            ptn(Sum - i, K - 1, lst + [i],total)
    ptn(Sum,K,[],total)
    return total


def fixed_locus(n, N):
    '''gives list of fixed points of Hilb^n1 x Hilb^n2 x ... x Hilb^nN, input (n1,n2,...,nN)'''
    total=[]
    ptns=partitions(n,N)
    for j in range(len(ptns)):
        ptn=ptns[j]
        lst=[]
        for i in range(N):
            lst+=[Partitions(ptn[i]).list()]
        total+=[p for p in itertools.product(*lst)]
    return total

def Nvir(Z):
    '''virtual normal bundle of quot scheme at (Z_1,...,Z_N)'''
    total=0
    for i in range(len(Z)):
        for j in range(len(Z)):
            total+=(1-character(Z[i],x=x^-1,y=y^-1)*(1-x)*(1-y))*character(Z[j],x=x,y=y)*(z[j]/z[i])
    return total


def total_chern_quot(Z,rank):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,y)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod((1+r) for r in roots)
    return total

def top_chern_quot(Z):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,y)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod((r) for r in roots)
    return total

def Chern_Num_quot(n,N,rank):
    total=0
    print(len(fixed_locus(n,N)))
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_chern_quot(Z,rank)
        total+=add
        print(add)
    return total

def DT_quot(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))
        total+=add
    return total

def total_segre_quot(Z,rank):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,y)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod(1/(1+r) for r in roots)
    return total

def Segre_Num_quot(n,N,rank):
    total=0
    print(len(fixed_locus(n,N)))
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_segre_quot(Z,rank)
        total+=add
        print(pretty(add))
        print("added")
    return total
def Segre_Num_quot_sym(n,N,rank):
    total=0
    print(len(fixed_locus(n,rank)))
    for Z in fixed_locus(n,rank):
        add=symmetrize(top_chern(-Nvir(Z)))*symmetrize(total_segre_quot(Z,N))
        total+=add
        print("added")
    return total


def determinant_quot(Z,n,rank):
    det=1
    for i in range(len(Z)):
        Q = character(Z[i],x,y)*(z[i])
        det *= determinant(Q)
    return (det^rank)*((prod(v[i] for i in range(rank)))^n)

def Verlinde_Num_quot(n,N,rank):
    total=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            determinant_quot(Z,n,rank)
        total+=add
    return total

def Verlinde_Num_quot_inv(n,N,rank):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            (1/determinant_quot(Z,n,rank))
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total


def deg0(f):
    if f==0: 
        return 0
    return degn(f(t=b*t,m=b*m,
                         l1=b*l1,l2=b*l2,l3=b*l3,l4=b*l4,l5=b*l5,
                         w1=b*w1,w2=b*w2,w3=b*w3,w4=b*w4,w5=b*w5),0)

def deg02(f):
    if f==0: 
        return 0
    return pretty(degn(f(t=b*t,m=b*m,w=b*w,v=b*v),0))

def set_exp(f):
    for i in range(len(KT_vars)):
        f=SR(f).subs(KT_ring.gens()[i]==exp(coho_ring.gens()[i]))
    return f
def simplify_weight(f):
    return f(l1=0,l2=l1,l3=2*l1,l4=3*l1,l5=4*l5)

def symmetrize(f):
    if f in ZZ:
        return f
    return f(l1=w1,l2=w2,l3=w3,l4=w4,l5=w5,w1=l1,w2=l2,w3=l3,w4=l4,w5=l5)


def symmetrizeK(f):
    if f in ZZ:
        return f
    return f(z1=v1,z2=v2,z3=v3,z4=v4,z5=v5,v1=z1,v2=z2,v3=z3,v4=z4,v5=z5)

def all_check(start,finish):
    with open("symmetry2d-segre-checked.txt") as f:
        lines = f.read().splitlines()
    if len(lines)==0:
        n,N,r=[0,0,0]
    else:
        last_checked=lines[len(lines)-1]
        n,N,r=[int(a) for a in last_checked.split(",")]
    for i in range(start,finish+1):
        for j in range(1,2):
            k=j
            n,N,r=[i,j,k]
            if k<j:
                continue
            if j<=k:
                print("checking segre sym: n="+str(n)+", N="+str(N)+", r="+str(r))
                
                S1=Segre_Num_quot(n,N,r)
                S2=Segre_Num_quot_sym(n,N,r)
                print("comparing")
                diff=S1*(-1)^(n*N)-S2*(-1)^(n*r)
                if diff==0:
                    writeline("symmetry2d-segre-checked.txt",str(n)+','+str(N)+','+str(r))
                    print("checked")
                else:
                    print("failed")
                              
        print("all checked for: n="+str(i))
                        
                    
def writeline(file,s):
    g = open(file, "a")
    g.write(s+"\n")
    g.close()
                
                
def clear_all_files():
    open('SV2d-checked.txt', 'w').close()
    open('symmetry2d-segre-checked.txt', 'w').close()
    open('symmetry2d-Verlinde-checked.txt', 'w').close()
    open('verlinde2d-out.txt', 'w').close()
    open('segre2d-out.txt', 'w').close()
    open('SV2d-out.txt', 'w').close()
            

def check_nek(N,nlimit):
    with open("nek2d-checked.txt") as f:
        lines = f.read().splitlines()
    for n in range(1,nlimit+1):
        print("checking n="+str(n)+", N="+str(N))
        if str(n)+','+str(N) in lines:
            print("n="+str(n)+", N="+str(N)+" already checked")
        else:
            a=Chern_Num_quot(n,N,N-1)
            b=(m+t)^n/factorial(n)/(m*t)^n
            if a==b:
                writeline("nek2d-checked.txt",str(n)+','+str(N))
                print("checked")
            else:
                print("failed")
                
def check_vanish(N,nlimit):
    with open("chern-vanish-checked.txt") as f:
        lines = f.read().splitlines()
    for n in range(1,nlimit+1):
        for r in range(0,N-1):
            print("checking n="+str(n)+", N="+str(N)+', r='+str(r))
            if str(n)+','+str(N)+','+str(r) in lines:
                print("n="+str(n)+", N="+str(N)+','+str(r)+" already checked")
            else:
                a=Chern_Num_quot(n,N,r)
                if a==0:
                    writeline("chern-vanish-checked.txt",str(n)+','+str(N)+','+str(r))
                    print("checked")
                else:
                    print("failed")
                    
def check_vanish2(N,nlimit):
    with open("verlinde-vanish-checked.txt") as f:
        lines = f.read().splitlines()
    for n in range(1,nlimit+1):
        for r in range(1,N):
            print("checking n="+str(n)+", N="+str(N)+', r='+str(r))
            if str(n)+','+str(N)+','+str(r) in lines:
                print("n="+str(n)+", N="+str(N)+','+str(r)+" already checked")
            else:
                a=Verlinde_Num_quot_inv(n,N,r)
                if a==0:
                    writeline("verlinde-vanish-checked.txt",str(n)+','+str(N)+','+str(r))
                    print("checked")
                else:
                    print("failed")

In [5]:
pretty(Chern_Num_quot(2,1,3))

2
(-2*t^5*l1^3 - 3*t^4*m*l1^3 - t^3*m^2*l1^3 + 6*t^4*l1^4 + 9*t^3*m*l1^4 + 3*t^2*m^2*l1^4 - 6*t^3*l1^5 - 9*t^2*m*l1^5 - 3*t*m^2*l1^5 + 2*t^2*l1^6 + 3*t*m*l1^6 + m^2*l1^6 - 2*t^5*l1^2*w1 - 3*t^4*m*l1^2*w1 - t^3*m^2*l1^2*w1 + 8*t^4*l1^3*w1 + 12*t^3*m*l1^3*w1 + 4*t^2*m^2*l1^3*w1 - 10*t^3*l1^4*w1 - 15*t^2*m*l1^4*w1 - 5*t*m^2*l1^4*w1 + 4*t^2*l1^5*w1 + 6*t*m*l1^5*w1 + 2*m^2*l1^5*w1 + 2*t^4*l1^2*w1^2 + 3*t^3*m*l1^2*w1^2 + t^2*m^2*l1^2*w1^2 - 4*t^3*l1^3*w1^2 - 6*t^2*m*l1^3*w1^2 - 2*t*m^2*l1^3*w1^2 + 2*t^2*l1^4*w1^2 + 3*t*m*l1^4*w1^2 + m^2*l1^4*w1^2 - 2*t^5*l1^2*w2 - 3*t^4*m*l1^2*w2 - t^3*m^2*l1^2*w2 + 8*t^4*l1^3*w2 + 12*t^3*m*l1^3*w2 + 4*t^2*m^2*l1^3*w2 - 10*t^3*l1^4*w2 - 15*t^2*m*l1^4*w2 - 5*t*m^2*l1^4*w2 + 4*t^2*l1^5*w2 + 6*t*m*l1^5*w2 + 2*m^2*l1^5*w2 - 2*t^5*l1*w1*w2 - 3*t^4*m*l1*w1*w2 - t^3*m^2*l1*w1*w2 + 10*t^4*l1^2*w1*w2 + 15*t^3*m*l1^2*w1*w2 + 5*t^2*m^2*l1^2*w1*w2 - 16*t^3*l1^3*w1*w2 - 24*t^2*m*l1^3*w1*w2 - 8*t*m^2*l1^3*w1*w2 + 8*t^2*l1^4*w1*w2 + 12*t*m*l1^4*w1*w2 + 4*m^2*l1^4*w1*w2 + 2

1/2*(l1^3*m + l1^3*t + 3*l1^2*m*t - 6*l1*m^2*t + 2*m^3*t - 6*l1*m*t^2 + 3*m^2*t^2 + 2*m*t^3 + l1^2*m*w1 + l1^2*t*w1 + 2*l1*m*t*w1 - 2*m^2*t*w1 - 2*m*t^2*w1 + l1^2*m*w2 + l1^2*t*w2 + 2*l1*m*t*w2 - 2*m^2*t*w2 - 2*m*t^2*w2 + l1*m*w1*w2 + l1*t*w1*w2 + m*t*w1*w2 + l1^2*m*w3 + l1^2*t*w3 + 2*l1*m*t*w3 - 2*m^2*t*w3 - 2*m*t^2*w3 + l1*m*w1*w3 + l1*t*w1*w3 + m*t*w1*w3 + l1*m*w2*w3 + l1*t*w2*w3 + m*t*w2*w3 + m*w1*w2*w3 + t*w1*w2*w3 + 3*l1^2*m + 3*l1^2*t + 6*l1*m*t - 6*m^2*t - 6*m*t^2 + 2*l1*m*w1 + 2*l1*t*w1 + 2*m*t*w1 + 2*l1*m*w2 + 2*l1*t*w2 + 2*m*t*w2 + m*w1*w2 + t*w1*w2 + 2*l1*m*w3 + 2*l1*t*w3 + 2*m*t*w3 + m*w1*w3 + t*w1*w3 + m*w2*w3 + t*w2*w3 + 3*l1*m + 3*l1*t + 3*m*t + m*w1 + t*w1 + m*w2 + t*w2 + m*w3 + t*w3 + m + t)*(l1 + w1 + 1)*(l1 + w2 + 1)*(l1 + w3 + 1)*(m + t)/(m^2*t^2)

In [9]:
pretty(Chern_Num_quot(2,1,1)(l1=0))

2
added
added


1/2*(m*t + m*w1 + t*w1 + m + t)*(m + t)*(w1 + 1)/(m^2*t^2)

In [10]:
pretty(Chern_Num_quot(3,1,1)(l1=0))

3
added
added
added


1/6*(2*m*t + m*w1 + t*w1 + m + t)*(m*t + m*w1 + t*w1 + m + t)*(m + t)*(w1 + 1)/(m^3*t^3)

In [ ]:
pretty(Chern_Num_quot(2,1,1))

In [13]:
pretty(Segre_Num_quot(2,2,1))

5
([], [2])
0
(x + 1)*z2/(x*z1)
0
-(x^2*y - x^2 - y)*(x + 1)/x
!!-(x^2*y*z1 - x^2*z1 - y*z1 - z2)*(x + 1)/(x*z1)
1/2*(m + 2*t)*(m + t)/((l1 - l2 + t)*(l1 - l2)*(l2 - t + w1 + 1)*(l2 + w1 + 1)*(m - t)*m*t^2)
added
([], [1, 1])
0
(y + 1)*z2/(y*z1)
0
-(x*y^2 - y^2 - x)*(y + 1)/y
!!-(x*y^2*z1 - y^2*z1 - x*z1 - z2)*(y + 1)/(y*z1)
-1/2*(2*m + t)*(m + t)/((l1 - l2 + m)*(l1 - l2)*(l2 - m + w1 + 1)*(l2 + w1 + 1)*(m - t)*m^2*t)
added
([1], [1])
-x*y + x + y
-(x*y - x - y)*z2/z1
-(x*y - x - y)*z1/z2
-x*y + x + y
!!-(x*y - x - y)*(z1 + z2)^2/(z1*z2)
-(l1 - l2 + m + t)*(l1 - l2 - m - t)*(m + t)^2/((l1 - l2 + m)*(l1 - l2 - m)*(l1 - l2 + t)*(l1 - l2 - t)*(l1 + w1 + 1)*(l2 + w1 + 1)*m^2*t^2)
added
([2], [])
-(x^2*y - x^2 - y)*(x + 1)/x
0
(x + 1)*z1/(x*z2)
0
!!-(x^2*y*z2 - x^2*z2 - y*z2 - z1)*(x + 1)/(x*z2)
1/2*(m + 2*t)*(m + t)/((l1 - l2 - t)*(l1 - l2)*(l1 - t + w1 + 1)*(l1 + w1 + 1)*(m - t)*m*t^2)
added
([1, 1], [])
-(x*y^2 - y^2 - x)*(y + 1)/y
0
(y + 1)*z1/(y*z2)
0
!!-(x*y^2*z2 - y^2*z2 - x*z2 - z1)

1/2*(l1*l2*m - l1*m^2 - l2*m^2 + m^3 + l1*l2*t - 3*l1*m*t - 3*l2*m*t + 3*m^2*t - l1*t^2 - l2*t^2 + 3*m*t^2 + t^3 + l1*m*w1 + l2*m*w1 - 2*m^2*w1 + l1*t*w1 + l2*t*w1 - 6*m*t*w1 - 2*t^2*w1 + m*w1^2 + t*w1^2 + l1*m + l2*m - 2*m^2 + l1*t + l2*t - 6*m*t - 2*t^2 + 2*m*w1 + 2*t*w1 + m + t)*(m + t)/((l1 - m + w1 + 1)*(l1 - t + w1 + 1)*(l1 + w1 + 1)*(l2 - m + w1 + 1)*(l2 - t + w1 + 1)*(l2 + w1 + 1)*m^2*t^2)

In [26]:
f=Segre_Num_quot(2,2,2)
degn(f(t=b*t,m=b*m,
                         l1=b*l1,l2=b*l2,l3=b*l3,l4=b*l4,l5=b*l5,
                         w1=b*w1,w2=b*w2,w3=b*w3,w4=b*w4,w5=b*w5),-1)

5
added
added
added
added
added


TypeError: ECL says: Console interrupt.

In [29]:
g=(f(l1=0,l2=0,l3=0,l4=0,l5=0,w1=0,w2=0,w3=0,w4=0,w5=0))

In [40]:
degn(g(t=b*t,m=b*m,
                         l1=b*l1,l2=b*l2,l3=b*l3,l4=b*l4,l5=b*l5,
                         w1=b*w1,w2=b*w2,w3=b*w3,w4=b*w4,w5=b*w5),0)

-18*(m^2 + 2*m*t + t^2)/(m*t)

In [61]:
f=Segre_Num_quot(2,2,2)
g=(f)
X=degn(g(t=b*t,m=b*m,
                         l1=b*l1,l2=b*l2,l3=b*l3,l4=b*l4,l5=b*l5,
                         w1=b*w1,w2=b*w2,w3=b*w3,w4=b*w4,w5=b*w5),-1)
X

5
added
added
added
added
added


TypeError: ECL says: Console interrupt.

In [69]:
F=Verlinde_Num_quot(6,1,2)
G=F
pretty(G).__str__()

'(x^42*y^42 + x^44*y^39 + 2*x^43*y^40 + 2*x^42*y^41 + 2*x^41*y^42 + 2*x^40*y^43 + x^39*y^44 + x^46*y^36 + 2*x^45*y^37 + 5*x^44*y^38 + 6*x^43*y^39 + 8*x^42*y^40 + 9*x^41*y^41 + 8*x^40*y^42 + 6*x^39*y^43 + 5*x^38*y^44 + 2*x^37*y^45 + x^36*y^46 + x^46*y^35 + 4*x^45*y^36 + 7*x^44*y^37 + 11*x^43*y^38 + 14*x^42*y^39 + 15*x^41*y^40 + 15*x^40*y^41 + 14*x^39*y^42 + 11*x^38*y^43 + 7*x^37*y^44 + 4*x^36*y^45 + x^35*y^46 + x^47*y^33 + 4*x^46*y^34 + 8*x^45*y^35 + 13*x^44*y^36 + 19*x^43*y^37 + 21*x^42*y^38 + 23*x^41*y^39 + 25*x^40*y^40 + 23*x^39*y^41 + 21*x^38*y^42 + 19*x^37*y^43 + 13*x^36*y^44 + 8*x^35*y^45 + 4*x^34*y^46 + x^33*y^47 + 2*x^47*y^32 + 5*x^46*y^33 + 10*x^45*y^34 + 15*x^44*y^35 + 17*x^43*y^36 + 17*x^42*y^37 + 18*x^41*y^38 + 16*x^40*y^39 + 16*x^39*y^40 + 18*x^38*y^41 + 17*x^37*y^42 + 17*x^36*y^43 + 15*x^35*y^44 + 10*x^34*y^45 + 5*x^33*y^46 + 2*x^32*y^47 + x^48*y^30 + 3*x^47*y^31 + 8*x^46*y^32 + 13*x^45*y^33 + 14*x^44*y^34 + 11*x^43*y^35 + 6*x^42*y^36 - x^41*y^37 - 7*x^40*y^38 - 7*x^39*y^39 - 7*x^38*y^40 - x^37*y^41 + 6*x^36*y^42 + 11*x^35*y^43 + 14*x^34*y^44 + 13*x^33*y^45 + 8*x^32*y^46 + 3*x^31*y^47 + x^30*y^48 + x^48*y^29 + 4*x^47*y^30 + 8*x^46*y^31 + 8*x^45*y^32 + 2*x^44*y^33 - 9*x^43*y^34 - 26*x^42*y^35 - 43*x^41*y^36 - 55*x^40*y^37 - 63*x^39*y^38 - 63*x^38*y^39 - 55*x^37*y^40 - 43*x^36*y^41 - 26*x^35*y^42 - 9*x^34*y^43 + 2*x^33*y^44 + 8*x^32*y^45 + 8*x^31*y^46 + 4*x^30*y^47 + x^29*y^48 + 2*x^48*y^28 + 6*x^47*y^29 + 7*x^46*y^30 + 2*x^45*y^31 - 12*x^44*y^32 - 35*x^43*y^33 - 62*x^42*y^34 - 84*x^41*y^35 - 108*x^40*y^36 - 119*x^39*y^37 - 124*x^38*y^38 - 119*x^37*y^39 - 108*x^36*y^40 - 84*x^35*y^41 - 62*x^34*y^42 - 35*x^33*y^43 - 12*x^32*y^44 + 2*x^31*y^45 + 7*x^30*y^46 + 6*x^29*y^47 + 2*x^28*y^48 + 2*x^48*y^27 + 4*x^47*y^28 + x^46*y^29 - 11*x^45*y^30 - 35*x^44*y^31 - 67*x^43*y^32 - 96*x^42*y^33 - 124*x^41*y^34 - 144*x^40*y^35 - 157*x^39*y^36 - 165*x^38*y^37 - 165*x^37*y^38 - 157*x^36*y^39 - 144*x^35*y^40 - 124*x^34*y^41 - 96*x^33*y^42 - 67*x^32*y^43 - 35*x^31*y^44 - 11*x^30*y^45 + x^29*y^46 + 4*x^28*y^47 + 2*x^27*y^48 + x^49*y^25 + 3*x^48*y^26 + 3*x^47*y^27 - 4*x^46*y^28 - 24*x^45*y^29 - 53*x^44*y^30 - 81*x^43*y^31 - 108*x^42*y^32 - 124*x^41*y^33 - 132*x^40*y^34 - 135*x^39*y^35 - 137*x^38*y^36 - 135*x^37*y^37 - 137*x^36*y^38 - 135*x^35*y^39 - 132*x^34*y^40 - 124*x^33*y^41 - 108*x^32*y^42 - 81*x^31*y^43 - 53*x^30*y^44 - 24*x^29*y^45 - 4*x^28*y^46 + 3*x^27*y^47 + 3*x^26*y^48 + x^25*y^49 + x^49*y^24 + 2*x^48*y^25 - 13*x^46*y^27 - 37*x^45*y^28 - 62*x^44*y^29 - 82*x^43*y^30 - 86*x^42*y^31 - 78*x^41*y^32 - 60*x^40*y^33 - 40*x^39*y^34 - 20*x^38*y^35 - 10*x^37*y^36 - 10*x^36*y^37 - 20*x^35*y^38 - 40*x^34*y^39 - 60*x^33*y^40 - 78*x^32*y^41 - 86*x^31*y^42 - 82*x^30*y^43 - 62*x^29*y^44 - 37*x^28*y^45 - 13*x^27*y^46 + 2*x^25*y^48 + x^24*y^49 + x^49*y^23 + 2*x^48*y^24 - 4*x^47*y^25 - 20*x^46*y^26 - 40*x^45*y^27 - 57*x^44*y^28 - 54*x^43*y^29 - 30*x^42*y^30 + 11*x^41*y^31 + 62*x^40*y^32 + 115*x^39*y^33 + 158*x^38*y^34 + 188*x^37*y^35 + 202*x^36*y^36 + 188*x^35*y^37 + 158*x^34*y^38 + 115*x^33*y^39 + 62*x^32*y^40 + 11*x^31*y^41 - 30*x^30*y^42 - 54*x^29*y^43 - 57*x^28*y^44 - 40*x^27*y^45 - 20*x^26*y^46 - 4*x^25*y^47 + 2*x^24*y^48 + x^23*y^49 + x^49*y^22 - 9*x^47*y^24 - 24*x^46*y^25 - 38*x^45*y^26 - 34*x^44*y^27 - 4*x^43*y^28 + 52*x^42*y^29 + 123*x^41*y^30 + 205*x^40*y^31 + 283*x^39*y^32 + 351*x^38*y^33 + 401*x^37*y^34 + 427*x^36*y^35 + 427*x^35*y^36 + 401*x^34*y^37 + 351*x^33*y^38 + 283*x^32*y^39 + 205*x^31*y^40 + 123*x^30*y^41 + 52*x^29*y^42 - 4*x^28*y^43 - 34*x^27*y^44 - 38*x^26*y^45 - 24*x^25*y^46 - 9*x^24*y^47 + x^22*y^49 + x^50*y^20 + x^49*y^21 - x^48*y^22 - 10*x^47*y^23 - 24*x^46*y^24 - 25*x^45*y^25 - x^44*y^26 + 53*x^43*y^27 + 129*x^42*y^28 + 216*x^41*y^29 + 304*x^40*y^30 + 384*x^39*y^31 + 457*x^38*y^32 + 507*x^37*y^33 + 543*x^36*y^34 + 550*x^35*y^35 + 543*x^34*y^36 + 507*x^33*y^37 + 457*x^32*y^38 + 384*x^31*y^39 + 304*x^30*y^40 + 216*x^29*y^41 + 129*x^28*y^42 + 53*x^27*y^43 - x^26*y^44 - 25*x^25*y^45 - 24*x^24*y^46 - 10*x^23*y^47 - x^22*y^

In [51]:
pretty(X)

28*(2*m^2 - 7*m*t + 2*t^2)*(m + t)^2/(m^2*t^2)

In [52]:
pretty(Y)

21/16*(32*m^2 - 157*m*t + 32*t^2)*(m + t)^2/(m^2*t^2)

In [50]:
pretty(X-Y)

7/16*(32*m^2 + 23*m*t + 32*t^2)*(m + t)^2/(m^2*t^2)

In [53]:
f=Segre_Num_quot(1,2,2)
g=(f(l1=0,l2=0,l3=0,l4=0,l5=0,w1=0,w2=0,w3=0,w4=0,w5=0))
X=degn(g(t=b*t,m=b*m,
                         l1=b*l1,l2=b*l2,l3=b*l3,l4=b*l4,l5=b*l5,
                         w1=b*w1,w2=b*w2,w3=b*w3,w4=b*w4,w5=b*w5),0)
pretty(X)

2
added
added


0

In [17]:
pretty(Chern_Num_quot(2,1,))

2
added
added


1/2*(l1^2*m + l1^2*t + 2*l1*m*t - 2*m^2*t - 2*m*t^2 + l1*m*w1 + l1*t*w1 + m*t*w1 + l1*m*w2 + l1*t*w2 + m*t*w2 + m*w1*w2 + t*w1*w2 + 2*l1*m + 2*l1*t + 2*m*t + m*w1 + t*w1 + m*w2 + t*w2 + m + t)*(l1 + w1 + 1)*(l1 + w2 + 1)*(m + t)/(m^2*t^2)

In [18]:
all_check(2,2)

checking segre sym: n=2, N=1, r=1
2
added
added
2
added
added
comparing
checked
checking segre sym: n=2, N=2, r=2
5
added
added
added
added
added
5
added
added
added
added
added
comparing
checked
checking segre sym: n=2, N=3, r=3
9
added
added
added
added
added
added
added
added
added
9
added
added
added
added
added
added
added
added
added
comparing
checked
all checked for: n=2


In [37]:
check_vanish2(2,8)

checking n=1, N=2, r=1
n=1, N=2,1 already checked
checking n=2, N=2, r=1
n=2, N=2,1 already checked
checking n=3, N=2, r=1
n=3, N=2,1 already checked
checking n=4, N=2, r=1
n=4, N=2,1 already checked
checking n=5, N=2, r=1
n=5, N=2,1 already checked
checking n=6, N=2, r=1
n=6, N=2,1 already checked
checking n=7, N=2, r=1
n=7, N=2,1 already checked
checking n=8, N=2, r=1
added1/185
added2/185
added3/185
added4/185
added5/185
added6/185
added7/185
added8/185
added9/185
added10/185
added11/185
added12/185
added13/185
added14/185
added15/185
added16/185
added17/185
added18/185
added19/185
added20/185
added21/185
added22/185
added23/185
added24/185
added25/185
added26/185
added27/185
added28/185
added29/185
added30/185
added31/185
added32/185
added33/185
added34/185
added35/185
added36/185
added37/185
added38/185
added39/185
added40/185
added41/185
added42/185
added43/185
added44/185
added45/185
added46/185
added47/185
added48/185
added49/185
added50/185
added51/185
added52/185
added53/185


In [38]:
check_vanish2(3,5)
check_vanish2(4,3)
check_vanish2(5,2)

checking n=1, N=3, r=1
added1/3
added2/3
added3/3
checked
checking n=1, N=3, r=2
added1/3
added2/3
added3/3
checked
checking n=2, N=3, r=1
added1/9
added2/9
added3/9
added4/9
added5/9
added6/9
added7/9
added8/9
added9/9
checked
checking n=2, N=3, r=2
added1/9
added2/9
added3/9
added4/9
added5/9
added6/9
added7/9
added8/9
added9/9
checked
checking n=3, N=3, r=1
added1/22
added2/22
added3/22
added4/22
added5/22
added6/22
added7/22
added8/22
added9/22
added10/22
added11/22
added12/22
added13/22
added14/22
added15/22
added16/22
added17/22
added18/22
added19/22
added20/22
added21/22
added22/22
checked
checking n=3, N=3, r=2
added1/22
added2/22
added3/22
added4/22
added5/22
added6/22
added7/22
added8/22
added9/22
added10/22
added11/22
added12/22
added13/22
added14/22
added15/22
added16/22
added17/22
added18/22
added19/22
added20/22
added21/22
added22/22
checked
checking n=4, N=3, r=1
added1/51
added2/51
added3/51
added4/51
added5/51
added6/51
added7/51
added8/51
added9/51
added10/51
added11/

In [39]:
check_vanish2(5,3)

checking n=1, N=5, r=1
n=1, N=5,1 already checked
checking n=1, N=5, r=2
n=1, N=5,2 already checked
checking n=1, N=5, r=3
n=1, N=5,3 already checked
checking n=1, N=5, r=4
n=1, N=5,4 already checked
checking n=2, N=5, r=1
n=2, N=5,1 already checked
checking n=2, N=5, r=2
n=2, N=5,2 already checked
checking n=2, N=5, r=3
n=2, N=5,3 already checked
checking n=2, N=5, r=4
n=2, N=5,4 already checked
checking n=3, N=5, r=1
added1/65
added2/65
added3/65
added4/65
added5/65
added6/65
added7/65
added8/65
added9/65
added10/65
added11/65
added12/65
added13/65


KeyboardInterrupt: 

In [ ]:
check_vanish(5,3)

In [9]:
check_vanish(3,5)

checking n=1, N=3, r=0
n=1, N=3,0 already checked
checking n=1, N=3, r=1
n=1, N=3,1 already checked
checking n=2, N=3, r=0
n=2, N=3,0 already checked
checking n=2, N=3, r=1
n=2, N=3,1 already checked
checking n=3, N=3, r=0
n=3, N=3,0 already checked
checking n=3, N=3, r=1
n=3, N=3,1 already checked
checking n=4, N=3, r=0
n=4, N=3,0 already checked
checking n=4, N=3, r=1
n=4, N=3,1 already checked
checking n=5, N=3, r=0
n=5, N=3,0 already checked
checking n=5, N=3, r=1
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added

In [10]:
check_vanish(4,3)

checking n=1, N=4, r=0
added
added
added
added
checked
checking n=1, N=4, r=1
added
added
added
added
checked
checking n=1, N=4, r=2
added
added
added
added
checked
checking n=2, N=4, r=0
added
added
added
added
added
added
added
added
added
added
added
added
added
added
checked
checking n=2, N=4, r=1
added
added
added
added
added
added
added
added
added
added
added
added
added
added
checked
checking n=2, N=4, r=2
added
added
added
added
added
added
added
added
added
added
added
added
added
added
checked
checking n=3, N=4, r=0
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
checked
checking n=3, N=4, r=1
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
adde

In [17]:
check_vanish(5,2)

checking n=1, N=5, r=0
n=1, N=5,0 already checked
checking n=1, N=5, r=1
n=1, N=5,1 already checked
checking n=1, N=5, r=2
n=1, N=5,2 already checked
checking n=1, N=5, r=3
n=1, N=5,3 already checked
checking n=2, N=5, r=0
n=2, N=5,0 already checked
checking n=2, N=5, r=1
n=2, N=5,1 already checked
checking n=2, N=5, r=2
n=2, N=5,2 already checked
checking n=2, N=5, r=3
n=2, N=5,3 already checked
